In [1]:
from transformers import GPT2TokenizerFast, GPT2Tokenizer
from copy import deepcopy
import numpy as np
from random import random
from time import time
def sum_masks(o_mask_size, masks):
    iy = 0
    ix = 0
    o_mask = np.zeros((o_mask_size, o_mask_size))

    for m in masks:
        ms = len(m)
        o_mask[iy:iy+ms, ix:ix+ms] += m
        o_mask[iy+ms:, ix:ix+ms] = m[-1]
        iy+=ms
        ix+=ms
    return o_mask


/home/maciej/anaconda3/envs/tokenizex/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

from time import time
time()

1720776876.2278259

In [3]:
from research.tokenizex.model.tokenizer import TokenizexTokenizer
tokenizex = TokenizexTokenizer()

ModuleNotFoundError: No module named 'research'

In [4]:
tokenizer_fast = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
vocab = tokenizer.get_vocab()
vocabi = {v:k for k, v in vocab.items()}
SRC = "asd_pol.txt"
with open(SRC, "r") as f:
    all_txt = f.read()
def get_imput(txt, length=512):
    bi = int(random()*(len(txt)-1000))
    return txt[bi:bi+length]

/home/maciej/anaconda3/envs/tokenizex/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: 'asd_pol.txt'

In [ ]:
(list(range(0,5)))

[0, 1, 2, 3, 4]

In [ ]:
from research.tokenizex.utils.datasets import get_processed_dataset

ds = get_processed_dataset(256, 256, "cuda", 0, 42, TokenizexTokenizer, 'wikibook', False, "train")

# for i in range(30000):
#     # if i == 3529:
#     print(i)  
#     li = i
#     processed_batch = ds.get_batch()

In [ ]:
batch = ds.get_batch()

/home/maciej/anaconda3/envs/randomllm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
t = batch.target_ids[0]
e = batch.input_ids[0]
m = batch.attention_mask[0]
p = batch.positions[0]

In [ ]:
tokenizer.decode(e[m[-1]])

"lly added sweeping strokes, outlines and incisive dabs to the imposing frame.<|endoftext|>`` derek told me that he hadn't seen sunlight in five hundred years.<|endoftext|>picked her up.<|endoftext|>`` i wasn't doing anything anyway,'' cole replied.<|endoftext|>and the genetic modifications will require "

In [ ]:
p[m[-1]]

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 32, 33,
        34, 35, 36, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 48, 49,
        50, 51, 52, 53, 54], device='cuda:0')

In [ ]:

from functools import cache


def tokenize_atoms(txt, tokenizer):
    atxt = "".join(tokenizer.tokenize(txt))
    atxt = list(atxt)
    eatxt = tokenizer.encode(atxt)
    return eatxt

c = 0

def lazy_past_tokens_gather(atokens, tokenizer): #dev no dobra, może tak być dla tokenów kłopotliwych, ale dla reszty już nie powinno
    global c #dev
    c+=1 #dev
    past_emb_pos = []
    for i in range(1, len(atokens)+1):
        past_emb_pos.append(atokens[:i])
    return past_emb_pos

def past_tokens_gather(atokens, tokenizer):
    past_emb_pos = []
    for i in range(1, len(atokens)+1):
        pdec = tokenizer.decode(atokens[:i])
        if '�' in pdec:
            return lazy_past_tokens_gather(atokens, tokenizer)
        penc = tokenizer.encode(pdec)
        past_emb_pos.append(penc)
    return past_emb_pos

def create_emb_pos_mask(emb_pos):
    res = []
    # mask = np.eye(len(emb_pos)).astype(bool)
    mask = np.eye(len(emb_pos))
    for embt, m in zip(emb_pos, mask):
        res.append(embt[-1])
        for ie in embt[:-1]:
            is_added = True if res[:-1] == [] else False
            for j, je in enumerate(res[:-1]):
                if ie == je:
                    m[j] = 1
                    is_added = True
                    break
            if not is_added:
                raise Exception("target len != input len")
    return np.array(res)[:,0], np.array(res)[:,1], mask

# @cache
def tokenizex_encode_full(txt, tokenizer):
    tenc = tokenize_atoms(txt, tokenizer)
    res = past_tokens_gather(tenc, tokenizer)
    g = [list(zip(e, np.arange(len(e)))) for e in res]
    e, p, m = create_emb_pos_mask(g)
    return e, p, m


In [ ]:
def tokenizex_encode(txt, tokenizer, atomizer):
    tokens = tokenizer.encode(txt)

    res_emb = []
    res_pos = []
    partial_masks = []
    pos_len = 0
    
    for t in tokens:
        emb, pos, mask = atomizer[t]
        res_emb.extend(emb)
        res_pos.extend(pos+pos_len)
        pos_len += pos[-1] + 1
        partial_masks.append(mask)   
    
    full_mask = sum_masks(len(res_emb), partial_masks)
    return np.array(res_emb), np.array(res_pos), full_mask


In [ ]:
import re


def remove_whitespace_before_dots_p(text):
    text = tokenizer.decode(tokenizer.encode(text))
    text = re.sub(r'\s+\.', '.', text)
    text = re.sub(r"\s+\'", "'", text)
    text = re.sub(r'\s+\,', ',', text)
    text = re.sub(r'\s+\!', '!', text)
    text = re.sub(r'\s+\?', '?', text)
    text = re.sub(r'\n+\n', '\n', text)
    return text

In [ ]:
def validate(txt, emb, pos, mask, tokenizer):
    txt = remove_whitespace_before_dots_p(txt)
    # dec = tokenizer.decode(emb[mask[-1].astype(bool)])
    # assert dec == txt # dev transformer has no order, token-po out of order

    lemb = tokenize_atoms(txt, tokenizer)
    assert len(lemb) == len(emb)

    # comp = pos[mask[-1].astype(bool)]
    # for a, b in zip(comp.astype(int), list(range(len(comp)))):
    #     assert a == b # dev transformer has no order, token-po out of order

In [ ]:
def tokenizex_w_encode(txt, tokenizer):
    res_emb = []
    res_pos = []
    partial_masks = []
    pos_len = 0
    
    txtw = [" " + e for e in txt.split(" ")]
    if len(txtw[0]) == 1:
        del txtw[0]
    else:
        txtw[0] = txtw[0][1:]
    
    for w in txtw:
        emb, pos, mask = tokenizex_encode_full(w, tokenizer)
        res_emb.extend(emb)
        res_pos.extend(pos+pos_len)
        pos_len += pos[-1] + 1
        partial_masks.append(mask)   
    full_mask = sum_masks(len(res_emb), partial_masks)
    return np.array(res_emb), np.array(res_pos), full_mask


In [ ]:
txt = " return np.array(res_emb), np.array(res_pos), full_mask"
txt = """  𡨸 𡨸żóąs"""
txt = """cial Maurice Rossel reported that conditions there were "almost normal".
2014 – Under the terms of UN Security C"""
txt = tokenizer.prepare_for_tokenization(txt)[0]
txt = remove_whitespace_before_dots_p(txt)
txt

'cial Maurice Rossel reported that conditions there were "almost normal".\n2014 – Under the terms of UN Security C'

In [ ]:
# txt = errorous
txt = """\n\nY"""
txt = '\n\nf\ngh\n\n\n\nSta'

In [ ]:
txt = remove_whitespace_before_dots_p(txt)

In [ ]:
[220, 77, 6, 83]
[[220], [299], [299, 6], [77, 470]]

[[220], [299], [299, 6], [77, 470]]

In [ ]:
tokenizer.decode([77, 470])

"n't"

In [ ]:
txt = " n't"
txt = remove_whitespace_before_dots_p(txt)
tenc = tokenize_atoms(txt, tokenizer)
print(tenc)
res = past_tokens_gather(tenc, tokenizer)
print(res)
g = [list(zip(e, np.arange(len(e)))) for e in res]
e, p, m = create_emb_pos_mask(g)
print(e)
print(m)
validate(txt, e, p, m, tokenizer)

[77, 6, 83]
[[77], [77, 6], [77, 470]]
[ 77   6 470]
[[1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]


In [ ]:
e, p, m = tokenizex_encode_full(txt, tokenizer)
validate(txt, e, p, m, tokenizer)

In [ ]:
ee, pp, mm = tokenizex_w_encode(txt, tokenizer)
# print(ee)
# print(mm)
validate(txt, ee, pp, mm, tokenizer)

In [ ]:
def tokenizer_def(txt, tokenizer):
    return tokenizer.encode(txt), None, None

def tokenizer_my1(txt, tokenizer):
    tenc = tokenize_atoms(txt, tokenizer)
    res = past_tokens_gather(tenc, tokenizer)
    g = [list(zip(e, np.arange(len(e)))) for e in res]
    e, p, m = create_emb_pos_mask(g)
    return e, p, m

def tokenizer_my2(txt, tokenizer):
    e, p, m= tokenizex_w_encode(txt, tokenizer)
    return e, p, m

In [ ]:
errorous = ""
def time_valid_tokenization(txt_data, tokenizer, def_tokenizer, C, lbl_val = False):
    st = 0
    global errorous
    for i in range(C):
        txt = get_imput(txt_data, 512)
        # txti = def_tokenizer.decode(def_tokenizer.encode(get_imput(txt_data, 112)))
        txt = def_tokenizer.prepare_for_tokenization(txt)[0]
        txt = remove_whitespace_before_dots_p(txt)
        t1 = time()
        errorous = txt
        emb, pos, mask = tokenizer(txt, def_tokenizer)
        st += time()-t1

        if lbl_val: 
            validate(txt, emb, pos, mask, def_tokenizer)

    return st


In [ ]:
txt = " n't"
txt = remove_whitespace_before_dots_p(txt)
emb, pos, mask = tokenizer_my2(txt, tokenizer)
validate(txt, emb, pos, mask, tokenizer)

In [ ]:
default = time_valid_tokenization(all_txt, tokenizer_def, tokenizer, 500, False)

In [ ]:
# my2 = time_valid_tokenization(all_txt, tokenizer_my2, tokenizer, 500, True)

In [ ]:
def tokenizex_fin(txt,_):
    return tokenizex.text_to_ids_pos_mask(txt)


In [ ]:
mytx = time_valid_tokenization(all_txt, tokenizex_fin, tokenizer, 50, True)

In [ ]:
llm_examples = 128*500000

print(f"mytx/default ratio {mytx/default}")
# print(f"my2/default ratio {my2/default}")


mytx/default ratio 9.868575189206975


In [ ]:
def sum_pos(poses):
    res = []
    res.extend(poses[0])
    sum_slit = 0
    for i in range(1, len(poses)):
        sum_slit += poses[i-1][-1] + 1
        res.extend(poses[i] + sum_slit)
    return np.array(res)

In [ ]:
t1 = "example text to tok\\n\n\n\n\n\n\\n\n\n\\n\\nn\enize n\nn\and, żan\n\n\rgon\n\n ołn\wkn\n\n\a źn\e wpłn\n\\n\ywa detokenize, ślóza."
t2 = " ... ż eloó ziomelo"
t3 = " n't"

e1, p1, m1 = tokenizex.text_to_ids_pos_mask(t1)
e2, p2, m2 = tokenizex.text_to_ids_pos_mask(t2)
e3, p3, m3 = tokenizex.text_to_ids_pos_mask(t3)
txt = t1+t2+t3
txt

emb = np.concatenate([e1,e2,e3])
pos = sum_pos([p1,p2,p3])
mlen = m1.shape[0]+m2.shape[0]+m3.shape[0]
mask = sum_masks(mlen, [m1, m2, m3])


In [ ]:
validate(txt, emb, pos, mask, tokenizer)

In [ ]:
from abc import abstractmethod
import random
from typing import Optional

from datasets import load_dataset, load_from_disk
import numpy as np


class AbstractDataset:
    def __init__(self, seed: Optional[int] = None):
        self.set_rng(seed)

    def set_rng(self, seed: Optional[int] = None):
        np_rng = np.random.default_rng(seed)
        py_rng = random.Random(seed)

        self.np_rng = np_rng
        self.py_rng = py_rng

    @abstractmethod
    def get_document(self) -> str:
        raise NotImplementedError()
    
    
class WikiBookDataset(AbstractDataset):
    def __init__(
        self,
        seed: Optional[int] = None,
        use_dummy_dataset: bool = False,
        split: str = "train",
    ):
        super().__init__(seed=seed)
        assert split in ["train", "eval"]
        self.split = split

        self.dataset_wiki = load_dataset(
            "wikipedia", f"20220301.{'simple' if use_dummy_dataset else 'en'}"
        )["train"]
        self.dataset_book = (
            load_dataset("bookcorpus")["train"]
            if not use_dummy_dataset
            else self.dataset_wiki
        )

        self.bookcorpus_chance = len(self.dataset_book) / len(self.dataset_wiki)

    def get_document(self) -> str:
        selector = self.py_rng.random()
        if selector < self.bookcorpus_chance:
            return self._get_random_book_example()
        else:
            return self._get_random_wiki_example()

    def _belongs_to_split(self, document_id: int) -> bool:
        eval_percentage = 5

        if self.split == "train":
            return hash(document_id) % 100 >= eval_percentage
        elif self.split == "eval":
            return hash(document_id) % 100 < eval_percentage
        else:
            raise ValueError("split must be either 'train' or 'eval'")

    def _get_random_book_example(self) -> str:
        doc_id = None
        while doc_id is None or not self._belongs_to_split(doc_id):
            doc_id = self.py_rng.randint(0, len(self.dataset_book) - 1)
        document = self.dataset_book[doc_id]
        return document["text"]

    def _get_random_wiki_example(self) -> str:
        doc_id = None
        while doc_id is None or not self._belongs_to_split(doc_id):
            doc_id = self.py_rng.randint(0, len(self.dataset_book) - 1)
        document = self.dataset_wiki[doc_id]
        return document["text"]

In [ ]:
errorous = ""
def time_valid_tokenization(wb_data, tokenizer, def_tokenizer, C, lbl_val = False):
    st = 0
    global errorous
    for i in range(C):
        txt = wb_data.get_document()[:512]
        # txti = def_tokenizer.decode(def_tokenizer.encode(get_imput(txt_data, 112)))
        txt = def_tokenizer.prepare_for_tokenization(txt)[0]
        txt = remove_whitespace_before_dots_p(txt)
        errorous = txt
        t1 = time()
        emb, pos, mask = tokenizer(txt, def_tokenizer)
        st += time()-t1

        if lbl_val: 
            validate(txt, emb, pos, mask, def_tokenizer)

    return st

In [ ]:
wb_data = WikiBookDataset(2137, False, split="train")

KeyboardInterrupt: 

In [ ]:
C = 10

In [ ]:
default = time_valid_tokenization(wb_data, tokenizer_def, tokenizer, C, False)

In [ ]:
my = time_valid_tokenization(wb_data, tokenizex_fin, tokenizer, C, True)

In [ ]:
my

In [ ]:
print(f"ratio my/default: {my/default}")

In [ ]:
errorous

In [ ]:
# txt = errorous
txt = " n't"
e, p, m = tokenizex.text_to_ids_pos_mask(txt)
validate(txt, e, p, m, tokenizer)

In [ ]:
tokenize_atoms(txt, tokenizer)

In [ ]:
e

In [ ]:
txt = " n't"
d = tokenizer.encode(txt)
tokenizer.decode(d)

In [ ]:
import unicodedata

# Iterate over all possible Unicode code points
for code_point in range(1118481):  # 0x110000 is 0x10FFFF + 1
    try:
        character = chr(code_point)
        # Filter out non-printable characters by their category
        if unicodedata.category(character)[0] != 'C':
            txt = f" {character} "
            d = tokenizer.encode(txt)
            if txt != tokenizer.decode(d):
                print(f";{txt};")
                print(f";{tokenizer.decode(d)};")
                print()
    except ValueError:
        # Skip invalid code points
        continue

In [ ]:
sl = 3
e = np.arange(7)
m = np.ones((7,7))
p = np.arange(7)
surpass = len(e) - sl
if surpass > 0:
    e = e[:-surpass]
    p = p[:-surpass]
    m = m[:-surpass, :-surpass]

e, p, m

In [ ]:
from attr import dataclass
from typing import List


@dataclass
class LLMExample(object):
    input_ids: List[int]
    target_ids: List[int]
    should_calculate_loss: List[
        int
    ]  # e.g. in BERT loss is not calculated over non-masked tokens

@dataclass
class TokenizexExample(LLMExample):
    positions: np.ndarray
    attention_mask: np.ndarray

In [ ]:
from torch import nn
import torch

emb = nn.Embedding(10, 5)

